<p style="font-size: 45px; text-align: center;"><b>Machine Learning in Finance II - final project</b></p>
<p style="font-size: 35px; text-align: center;"><b>Forecasting delays in delivery time - Brazilian E-commerce</b></p>
<p style="font-size: 20px; text-align: center;"><b>XGBoost - choosing the best model</b></p>

Author: Jakub Pyszniak

Notebook 3

# Analysis overview

> **In this section we will choose the best XGBoost (regressor) model to predict order delays**

> **We perform the necessary Cross-Validation**

# Libraries

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectFromModel
from scipy import stats
import math
import time
from scipy.stats import randint, uniform, loguniform
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error

# Visual set-up
pd.set_option("display.max_columns", 60)

# XGBoost
import xgboost as xgb
from xgboost import XGBRegressor

# Loading data & choosing the best features

We can use the feature ranking from earlier to limit our feature ("X") set size from 30+ to 10 or 15 best

In [23]:
# These data will be used for CV and choosing the best models
df_train = pd.read_csv("4.train_and_test/df_train.csv")

# We will use this set to test the forecasting power of our best models (final comparisons)
df_test = pd.read_csv("4.train_and_test/df_test.csv")

# Our feature ranking dataset
fr = pd.read_excel("3.feature_ranking/feature_ranking.xlsx")

In [24]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73180 entries, 0 to 73179
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   delay_days                     73180 non-null  float64
 1   order_item_id                  73180 non-null  int64  
 2   total_price                    73180 non-null  float64
 3   total_freight                  73180 non-null  float64
 4   order_value_check              73180 non-null  float64
 5   is_payment_sequential_1        73180 non-null  int64  
 6   payment_type_boleto            73180 non-null  int64  
 7   payment_type_credit_card       73180 non-null  int64  
 8   payment_type_debit_card        73180 non-null  int64  
 9   payment_type_voucher           73180 non-null  int64  
 10  installments_boleto            73180 non-null  int64  
 11  installments_credit_card       73180 non-null  int64  
 12  installments_debit_card        73180 non-null 

,delay_days,order_item_id,total_price,total_freight,order_value_check,is_payment_sequential_1,payment_type_boleto,payment_type_credit_card,payment_type_debit_card,payment_type_voucher,installments_boleto,installments_credit_card,installments_debit_card,installments_voucher,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_state,customer_state,cust_sell_same_state,customer_lat,customer_lng,seller_lat,seller_lng,cust_sell_distance_km,order_purchase_month,order_delivery_month,order_year_2016,order_year_2017,order_year_2018,delivery_year_2016,delivery_year_2017,delivery_year_2018
0,-21.0,1,19.99,14.10,34.09,1,0,1,0,0,0,1,0,0,48.0,575.0,1,100.0,20.0,15.0,15.0,43,21,18,0,-22.761992,-43.450873,-23.665703,-46.518082,329.149657,6,6,0,1,0,0,1,0
1,-4.0,1,72.90,19.70,92.60,1,0,1,0,0,0,1,0,0,37.0,360.0,1,650.0,45.0,15.0,25.0,67,14,25,0,-21.679558,-49.762053,-23.179392,-50.634922,189.372236,8,8,0,0,1,0,0,1
2,-38.0,1,50.90,15.57,66.47,1,0,1,0,0,0,1,0,0,60.0,473.0,1,600.0,30.0,4.0,20.0,39,21,18,0,-22.449744,-43.474330,-21.766477,-48.831547,557.068979,2,2,0,1,0,0,1,0
3,-8.0,1,199.90,14.23,214.13,1,0,1,0,0,0,5,0,0,60.0,233.0,1,2600.0,41.0,8.0,36.0,7,21,25,1,-23.640572,-46.570773,-22.716839,-47.657366,151.282904,6,6,0,0,1,0,0,1
4,-12.0,2,44.00,35.26,79.26,1,0,1,0,0,0,7,0,0,58.0,1623.0,1,200.0,26.0,10.0,22.0,7,5,18,0,-22.983577,-43.220723,-16.692331,-49.268016,942.794225,3,3,0,0,1,0,0,1


In [25]:
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18295 entries, 0 to 18294
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   delay_days                     18295 non-null  float64
 1   order_item_id                  18295 non-null  int64  
 2   total_price                    18295 non-null  float64
 3   total_freight                  18295 non-null  float64
 4   order_value_check              18295 non-null  float64
 5   is_payment_sequential_1        18295 non-null  int64  
 6   payment_type_boleto            18295 non-null  int64  
 7   payment_type_credit_card       18295 non-null  int64  
 8   payment_type_debit_card        18295 non-null  int64  
 9   payment_type_voucher           18295 non-null  int64  
 10  installments_boleto            18295 non-null  int64  
 11  installments_credit_card       18295 non-null  int64  
 12  installments_debit_card        18295 non-null 

,delay_days,order_item_id,total_price,total_freight,order_value_check,is_payment_sequential_1,payment_type_boleto,payment_type_credit_card,payment_type_debit_card,payment_type_voucher,installments_boleto,installments_credit_card,installments_debit_card,installments_voucher,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_state,customer_state,cust_sell_same_state,customer_lat,customer_lng,seller_lat,seller_lng,cust_sell_distance_km,order_purchase_month,order_delivery_month,order_year_2016,order_year_2017,order_year_2018,delivery_year_2016,delivery_year_2017,delivery_year_2018
0,-15.0,1,108.0,16.52,124.52,1,0,1,0,0,0,1,0,0,58.0,3006.0,2,1000.0,53.0,8.0,18.0,72,15,10,0,-21.248980,-44.998179,-22.874599,-43.477731,239.212282,2,2,0,0,1,0,0,1
1,-2.0,1,78.0,7.80,85.80,1,0,1,0,0,0,2,0,0,59.0,319.0,4,250.0,16.0,2.0,20.0,72,21,25,1,-23.657047,-46.774874,-23.651115,-46.755211,2.108617,11,11,0,1,0,0,1,0
2,-19.0,1,199.9,15.15,215.05,1,0,1,0,0,0,5,0,0,55.0,623.0,1,337.0,16.0,13.0,13.0,72,21,18,0,-22.704280,-43.571287,-22.828655,-47.267296,379.200398,6,6,0,1,0,0,1,0
3,-17.0,1,69.0,19.45,88.45,1,0,1,0,0,0,6,0,0,51.0,324.0,1,900.0,42.0,8.0,37.0,7,21,18,0,-22.747569,-43.488349,-21.766477,-48.831547,560.549267,6,6,0,0,1,0,0,1
4,-5.0,1,44.9,9.42,54.32,1,1,0,0,0,1,0,0,0,51.0,1118.0,5,400.0,18.0,8.0,14.0,49,21,25,1,-23.663579,-46.617176,-23.198860,-47.293346,86.193580,3,3,0,0,1,0,0,1


In [26]:
fr.rename(columns={"Unnamed: 0": "feature"}, inplace=True)

fr = fr.sort_values(by="boruta_rank")

fr.info()
fr.head(40)

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 24 to 7
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   feature      37 non-null     object 
 1   mi_score     37 non-null     float64
 2   boruta_rank  37 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ KB


,feature,mi_score,boruta_rank
24,customer_lat,0.046841,1
25,customer_lng,0.046367,1
27,seller_lng,0.056669,1
28,cust_sell_distance_km,0.052615,1
30,order_delivery_month,0.062269,1
29,order_purchase_month,0.061673,1
21,seller_state,0.020286,1
31,order_year_2016,0.008304,1
23,cust_sell_same_state,0.036475,1
34,delivery_year_2016,0.008570,1


In [28]:
fr = fr.set_index("feature")

We will create arrays with the best features so that we can pick them later from our dataframes

In [29]:
br10_feat = fr.iloc[0:10].index.tolist()
print("10 best:\n", br10_feat)

br15_feat = fr.iloc[0:15].index.tolist()
print("15 best:\n", br15_feat)

br20_feat = fr.iloc[0:20].index.tolist()
print("20 best:\n", br20_feat)

10 best:
 ['customer_lat', 'customer_lng', 'seller_lng', 'cust_sell_distance_km', 'order_delivery_month', 'order_purchase_month', 'seller_state', 'order_year_2016', 'cust_sell_same_state', 'delivery_year_2016']
15 best:
 ['customer_lat', 'customer_lng', 'seller_lng', 'cust_sell_distance_km', 'order_delivery_month', 'order_purchase_month', 'seller_state', 'order_year_2016', 'cust_sell_same_state', 'delivery_year_2016', 'order_year_2018', 'delivery_year_2018', 'delivery_year_2017', 'order_year_2017', 'total_freight']
20 best:
 ['customer_lat', 'customer_lng', 'seller_lng', 'cust_sell_distance_km', 'order_delivery_month', 'order_purchase_month', 'seller_state', 'order_year_2016', 'cust_sell_same_state', 'delivery_year_2016', 'order_year_2018', 'delivery_year_2018', 'delivery_year_2017', 'order_year_2017', 'total_freight', 'seller_lat', 'product_name_length', 'product_weight_g', 'product_height_cm', 'product_description_length']


## X and y test/train

In [36]:
# build X / y splits using only the selected features

X10_train = df_train.loc[:, br10_feat].copy()
X15_train = df_train.loc[:, br15_feat].copy()
X20_train = df_train.loc[:, br20_feat].copy()
y_train = df_train.loc[:, "delay_days"].copy()

X10_test  = df_test.loc[:, br10_feat].copy()
X15_test  = df_test.loc[:, br15_feat].copy()
X20_test  = df_test.loc[:, br20_feat].copy()
y_test  = df_test.loc[:, "delay_days"].copy()

In [40]:
X10_train.info(), X15_train.info(), X20_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73180 entries, 0 to 73179
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_lat           73180 non-null  float64
 1   customer_lng           73180 non-null  float64
 2   seller_lng             73180 non-null  float64
 3   cust_sell_distance_km  73180 non-null  float64
 4   order_delivery_month   73180 non-null  int64  
 5   order_purchase_month   73180 non-null  int64  
 6   seller_state           73180 non-null  int64  
 7   order_year_2016        73180 non-null  int64  
 8   cust_sell_same_state   73180 non-null  int64  
 9   delivery_year_2016     73180 non-null  int64  
dtypes: float64(4), int64(6)
memory usage: 5.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73180 entries, 0 to 73179
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0  

(None, None, None)

In [41]:
X10_test.info(), X15_test.info(), X20_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18295 entries, 0 to 18294
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_lat           18295 non-null  float64
 1   customer_lng           18295 non-null  float64
 2   seller_lng             18295 non-null  float64
 3   cust_sell_distance_km  18295 non-null  float64
 4   order_delivery_month   18295 non-null  int64  
 5   order_purchase_month   18295 non-null  int64  
 6   seller_state           18295 non-null  int64  
 7   order_year_2016        18295 non-null  int64  
 8   cust_sell_same_state   18295 non-null  int64  
 9   delivery_year_2016     18295 non-null  int64  
dtypes: float64(4), int64(6)
memory usage: 1.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18295 entries, 0 to 18294
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0  

(None, None, None)

In [34]:
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 73180 entries, 0 to 73179
Series name: delay_days
Non-Null Count  Dtype  
--------------  -----  
73180 non-null  float64
dtypes: float64(1)
memory usage: 571.8 KB


In [35]:
y_test.info()

<class 'pandas.core.series.Series'>
RangeIndex: 18295 entries, 0 to 18294
Series name: delay_days
Non-Null Count  Dtype  
--------------  -----  
18295 non-null  float64
dtypes: float64(1)
memory usage: 143.1 KB


# Model selection

A standard GridSearchCV might be too taxing for an XGBoost model if we want to try out a variety of hyperparameters. We will use RandomizedSearchCV as a basis.

> Our RandomizedSearchCV set-up

In [ ]:
def cv_my_random_xgb(
    X_train, y_train, X_test, y_test,
    param_distributions=None,
    n_iter=60,
    cv=5,
    random_state=42,
    show_results=False
):
    # ----------------------------
    # 0) Start a high-resolution timer so we can report total runtime at the end
    # ----------------------------
    start = time.perf_counter()

    # ----------------------------
    # 1) Define the cross-validation splitter
    #    - shuffle=True helps avoid fold bias if your rows are ordered in any way
    #           - they already should be shuffled but we use this as a precaution
    # ----------------------------
    cv_split = KFold(n_splits=cv, shuffle=True, random_state=random_state)

    # ----------------------------
    # 2) Define the base XGBoost model (regression)
    #    - objective="reg:squarederror" is standard for regression RMSE/MAE tasks
    #    - tree_method="hist" tends to be faster on CPU for medium/large datasets
    #       - alternatively we could use "exact" for the original tree method or "approx"
    #       - we want to, however, check out many different HP so we need to make sure models run fast
    #    - n_jobs=-1 uses all CPU cores
    #    - verbosity=0 silences XGBoost training output
    # ----------------------------
    xgb = XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",
        n_jobs=-1,
        random_state=random_state,
        verbosity=0
    )

    # ----------------------------
    # 3) Define hyperparameter search space ("param_distributions")
    #    If user does not pass one, we use a reasonable default.
    #
    #    NOTE: 1) RandomizedSearchCV will sample n_iter random combinations from these distributions.
    #    NOTE: 2) We can input values directly as arrays or we can assign distributions to sample from
    # ----------------------------
    if param_distributions is None:
        param_distributions = {
            # Number of boosting rounds (trees). Base learner is a tree
            #"n_estimators": randint(100, 1000),

            # Step size shrinkage. Log-uniform is typical since good values vary by orders of magnitude.
            "learning_rate": loguniform(0.01, 0.2), # generally, we are looking for small values

            # Controls tree complexity (how deep they go)
            "max_depth": randint(2, 10),

            # Minimum sum of instance weight needed (hessian) in a child to split a node (regularizes tree growth)
            "min_child_weight": loguniform(0.5, 50.0),

            # Row subsampling  - we won't choose all the rows for training each particular tree (regularization + faster training)
            "subsample": uniform(0.6, 0.4),  # from 0.6  to 1.0

            # Column subsampling per tree - similarily we will choose a random percentage of columns (regularization + faster training)
            "colsample_bytree": uniform(0.6, 0.4),  # from 0.6  to 1.0

            # Minimum loss reduction to make a further partition (regularization)
            "gamma": loguniform(1e-8, 5.0),

            # L1 regularization (sparsity)
            "reg_alpha": loguniform(1e-8, 10.0),

            # L2 regularization (weight shrinkage)
            "reg_lambda": loguniform(0.5, 50.0),
        }

    # ----------------------------
    # 4) Set up RandomizedSearchCV
    #    - scoring uses neg_root_mean_squared_error (sklearn convention: higher is better, hence negative)
    #    - cv=cv_split ensures the exact splitter we defined is used
    #    - refit=True refits the best found model on the FULL training set at the end
    # ----------------------------
    search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_distributions,
        n_iter=n_iter,
        scoring="neg_root_mean_squared_error",
        cv=cv_split,
        refit=True,
        return_train_score=True,
        n_jobs=-1,
        random_state=random_state,
        verbose=0
    )

    # ----------------------------
    # 5) Run hyperparameter tuning on the TRAINING data only
    #    This will internally do:
    #      - sample n_iter parameter combinations
    #      - for each combination, run cross-validation with 'cv' folds
    #      - select the combination with the best mean CV score
    #      - refit the best model on the full X_train/y_train (because refit=True)
    # ----------------------------
    search.fit(X_train, y_train)

    # After fit: best_estimator_ is the refit model trained on the full training set
    best_model = search.best_estimator_

    # ----------------------------
    # 6) Final evaluation on the TEST set (unseen during tuning - obvious but crucial to not forget during CV set-ups)
    # ----------------------------
    y_pred = best_model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae  = mean_absolute_error(y_test, y_pred)
    r2   = r2_score(y_test, y_pred)

    # ----------------------------
    # 7) Stop timer and compute elapsed time (for convenience)
    # ----------------------------
    elapsed_sec = time.perf_counter() - start

    # ----------------------------
    # 8) Optional printing (off by default)
    # ----------------------------
    if show_results:
        print("Best params:", search.best_params_)
        print("Best CV RMSE:", -search.best_score_)  # negate because scoring is negative RMSE
        print("TEST RMSE:", rmse)
        print("TEST MAE:", mae)
        print("TEST R²:", r2)
        print(f"Elapsed time: {elapsed_sec:.2f} seconds")

    # ----------------------------
    # 9) Return objects + metrics so you can inspect results later
    # ----------------------------
    return {
        "search": search,                     # full RandomizedSearchCV object (cv_results_, best_params_, etc.)
        "best_model": best_model,             # the refit best estimator trained on full X_train/y_train
        "best_cv_rmse": -search.best_score_,  # best mean CV RMSE (converted back to positive)
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "elapsed_sec": elapsed_sec
    }


**10 variables**

In [47]:
param_distributions = {
            #"n_estimators": randint(100, 1000),
            "learning_rate": loguniform(0.01, 0.2), # generally, we are looking for small values
            "max_depth": randint(2, 10),
            "min_child_weight": loguniform(0.5, 50.0),
            "subsample": uniform(0.6, 0.4),  # from 0.6  to 1.0
            "colsample_bytree": uniform(0.6, 0.4),  # from 0.6  to 1.0
            "gamma": loguniform(1e-8, 5.0),
            "reg_alpha": loguniform(1e-8, 10.0),
            "reg_lambda": loguniform(0.5, 50.0),
        }


xgb10vars_RS = cv_my_random_xgb(
    X10_train, y_train, X10_test, y_test,
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    random_state=42,
    show_results=True
)

Best params: {'colsample_bytree': np.float64(0.7396838298450643), 'gamma': np.float64(0.02065697267209068), 'learning_rate': np.float64(0.14694924450340516), 'max_depth': 9, 'min_child_weight': np.float64(3.2209668757412535), 'reg_alpha': np.float64(0.9770817213609811), 'reg_lambda': np.float64(25.166736492345343), 'subsample': np.float64(0.9742539976883791)}
Best CV RMSE: 7.511229251471095
TEST RMSE: 7.404489309153397
TEST MAE: 5.265760271008635
TEST R²: 0.48852922186404313
Elapsed time: 79.25 seconds


> **FINE-TUNE numebr of rounds with early stopping later with xgb.cv**